#CAD PROJECT

Baseline model based on:
https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#initialize-and-reshape-the-networks


Made by MAIA team: Jaime, Ahmed & "Prem"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* In **finetuning**, we start with a pretrained model and update *all* of the model’s parameters for our new task, in essence retraining the whole model.
* In **feature extraction** we start with a pretrained model and only update the final layer weights from which we derive predictions. It is called feature extraction because we use the pretrained CNN as a fixed feature-extractor, and only change the output layer.

In general both transfer learning methods follow the same few steps:

-  Initialize the pretrained model
-  Reshape the final layer(s) to have the same number of outputs as the
   number of classes in the new dataset
-  Define for the optimization algorithm which parameters we want to
   update during training
-  Run the training step




In [0]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import random
import pandas as pd
import sys
import os
import copy
%matplotlib inline

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

!pip install barbar

PyTorch Version:  1.3.1
Torchvision Version:  0.4.2


In [0]:
path_folder='/content/drive/My Drive/Colab Notebooks/CAD Project/'
sys.path.insert(1, path_folder) # Import py files in the folder

Inputs
------


The other inputs are as follows: ``num_classes`` is the number of
classes in the dataset, ``batch_size`` is the batch size used for
training and may be adjusted according to the capability of your
machine, ``num_epochs`` is the number of training epochs we want to run,
and ``feature_extract`` is a boolean that defines if we are finetuning
or feature extracting. If ``feature_extract = False``, the model is
finetuned and all model parameters are updated. If
``feature_extract = True``, only the last layer parameters are updated,
the others remain fixed.




In [0]:
###### SELECT CHALLANGE TYPE ##########

CHALLANGE_TYPE = {
    1: "Dermo Challenge",
    2: "Histopathology Challenge",
}
CHALLANGE_TYPE_INDEX = 2;

CHALLANGE_TYPE_STR =CHALLANGE_TYPE.get(CHALLANGE_TYPE_INDEX, "Invalid normalization type")
print(CHALLANGE_TYPE_STR)

###### CHANGE THIS PARAMETERS ###########
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
MODEL_NAME = "resnext50"
LR=1e-4
# EXPERIMENT NAMES
EXPERIMENT_NAME='Train_complete'
SUB_EXPERIMENT_NAME = MODEL_NAME+str(LR)

# Batch size for training (change depending on how much memory you have)
BATCH_SIZE = 32

# Number of epochs to train for 
NUM_EPOCHS = 16


PREPROCESS_DATA = False
AUGMENTATION_ENABLE = True

if(CHALLANGE_TYPE_INDEX == 1):
  INPUT_SIZE=225
if(CHALLANGE_TYPE_INDEX == 2):
  INPUT_SIZE=96

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
FEATURE_EXTRACT= False
USE_PRETRAINED = True


###### CHANGE THIS PARAMETERS ###########



Histopathology Challenge


In [0]:
options = {}
options['train_split']  = 0.2
options["input_size"]=INPUT_SIZE
options["challenge_dir"]=os.path.join(path_folder,CHALLANGE_TYPE_STR+"/")
if(PREPROCESS_DATA):
  options["train_dir"]=os.path.join(options["challenge_dir"],"train_pre/")
  options["val_dir"]=os.path.join(options["challenge_dir"],"val_pre/")
  options["test_dir"]=os.path.join(options["challenge_dir"],"test_pre/")
else:
  options["train_dir"]=os.path.join(options["challenge_dir"],"train/")
  options["val_dir"]=os.path.join(options["challenge_dir"],"val/")
  options["test_dir"]=os.path.join(options["challenge_dir"],"test/")
if(CHALLANGE_TYPE_INDEX == 1):
  options["experiments_dir"]=os.path.join(path_folder,"Dermo_Experiments")
if(CHALLANGE_TYPE_INDEX == 2):
  options["experiments_dir"]=os.path.join(path_folder,"Histo_Experiments")
options["models_dir"]=os.path.join(options["experiments_dir"],EXPERIMENT_NAME)
options["batch_size"]=BATCH_SIZE

# Number of classes in the dataset
num_classes = 2

Initialize and Reshape the Networks
-----------------------------------



In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import random
import pandas as pd
import sys
import os
import copy

def set_parameter_requires_grad(model, feature_extracting):
    """
     
    This helper function sets the ``.requires_grad`` attribute of the
    parameters in the model to False when we are feature extracting. By
    default, when we load a pretrained model all of the parameters have
    ``.requires_grad=True``, which is fine if we are training from scratch
    or finetuning. However, if we are feature extracting and only want to
    compute gradients for the newly initialized layer then we want all of
    the other parameters to not require gradients. This will make more sense
    later.
    """
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            

In [0]:
from MAIA_model import initialize_model
# Initialize the model for this run
model_ft = initialize_model(MODEL_NAME, num_classes, FEATURE_EXTRACT, use_pretrained=USE_PRETRAINED)

# Print the model we just instantiated
print(model_ft)

model_ft.to('cuda')

from torchsummary import summary
summary(model_ft, (3,INPUT_SIZE, INPUT_SIZE))

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:02<00:00, 41.5MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

Load Data
---------

Now that we know what the input size must be, we can initialize the data
transforms, image datasets, and the dataloaders. Notice, the models were
pretrained with the hard-coded normalization values, as described
`here <https://pytorch.org/docs/master/torchvision/models.html>`__.




In [0]:
from MAIA_Loader import get_train_valid_loader_AUG, get_val_loader,get_train_loader_AUG

data_loader_train=get_train_loader_AUG(data_dir=options["train_dir"],batch_size= options["batch_size"],augment=AUGMENTATION_ENABLE, input_size= options["input_size"],challenge_type=CHALLANGE_TYPE_INDEX)
data_loader_val=get_train_loader_AUG(data_dir=options["val_dir"],batch_size= options["batch_size"],augment=AUGMENTATION_ENABLE, input_size= options["input_size"],challenge_type=CHALLANGE_TYPE_INDEX)

dataset_complete_train=torch.utils.data.ConcatDataset((data_loader_train.dataset,data_loader_val.dataset))


dataloader_complete_train = torch.utils.data.DataLoader(
    dataset_complete_train, batch_size=options["batch_size"], shuffle=True)

len(dataloader_complete_train.dataset)

29494

Create the Optimizer
--------------------

Now that the model structure is correct, the final step for finetuning
and feature extracting is to create an optimizer that only updates the
desired parameters. Recall that after loading the pretrained model, but
before reshaping, if ``feature_extract=True`` we manually set all of the
parameter’s ``.requires_grad`` attributes to False. Then the
reinitialized layer’s parameters have ``.requires_grad=True`` by
default. So now we know that *all parameters that have
.requires_grad=True should be optimized.* Next, we make a list of such
parameters and input this list to the ADAM algorithm constructor.

To verify this, check out the printed parameters to learn. When
finetuning, this list should be long and include all of the model
parameters. However, when feature extracting this list should be short
and only include the weights and biases of the reshaped layers.




In [0]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if FEATURE_EXTRACT:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(params_to_update, lr=LR)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.0.conv3.weight
	 layer1.0.bn3.weight
	 layer1.0.bn3.bias
	 layer1.0.downsample.0.weight
	 layer1.0.downsample.1.weight
	 layer1.0.downsample.1.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.1.conv3.weight
	 layer1.1.bn3.weight
	 layer1.1.bn3.bias
	 layer1.2.conv1.weight
	 layer1.2.bn1.weight
	 layer1.2.bn1.bias
	 layer1.2.conv2.weight
	 layer1.2.bn2.weight
	 layer1.2.bn2.bias
	 layer1.2.conv3.weight
	 layer1.2.bn3.weight
	 layer1.2.bn3.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.conv3.weight
	 layer2.0.bn3.weight
	 layer2.0.bn3.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight

Train the model
--------------------------------

The next step is to setup the loss for the model, then run the
training and validation function for the set number of epochs. Notice,
depending on the number of epochs this step may take a while on a CPU.
Also, the default learning rate is not optimal for all of the models, so
to achieve maximum accuracy it would be necessary to tune for each model
separately.




In [0]:
from MAIA_model import train_complete_model

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

if not os.path.isdir(options["models_dir"]):
        print("Creating Experiment Folder")
        os.mkdir(options["models_dir"])

save_model_path= os.path.join(options['models_dir'], SUB_EXPERIMENT_NAME )

# Train the model
try:
  training_metrics = train_complete_model(device, model_ft,  dataloader_complete_train, criterion, optimizer_ft, num_epochs=NUM_EPOCHS,save_model_path=save_model_path,is_inception=(MODEL_NAME=="inception"))
except KeyboardInterrupt:
    pass

# Save results
df_training = pd.DataFrame(training_metrics, columns=["train_loss","train_acc","val_loss"," val_acc"],index=list(range(1,NUM_EPOCHS+1)))
df_training.to_excel(options["models_dir"]+'/'+MODEL_NAME+".xlsx")
df_training.describe().T

Epoch 1/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.3116 Acc: 0.8691

Epoch 2/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.2262 Acc: 0.9111

Epoch 3/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.1998 Acc: 0.9241

Epoch 4/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.1810 Acc: 0.9294

Epoch 5/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.1730 Acc: 0.9350

Epoch 6/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.1664 Acc: 0.9389

Epoch 7/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.1551 Acc: 0.9400

Epoch 8/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.1444 Acc: 0.9461

Epoch 9/16
----------
29494/29494: [===============================>] - ETA 0.2s
Loss: 0.1417 Acc: 0.9472

Epoch 10/16
----------
29494/29494: [

ValueError: ignored